# Logistische Regression

In diesem Kapitel implementieren wir mit PyTorch ein Modell für die **Logistische Regression**, das zwar in anderen Bibliotheken direkt integriert ist (sci-kit-leaern), uns aber wichtige Einblicke für das Training von deep-neural nets liefert.

In [64]:
# Module für die Datenanalyse und Visualisierung
import pandas as pd
import matplotlib.pyplot as plt

# Pytorch imports, die wir benötigen

import torch
from torch import nn # wir werden Methoden von nn.Module benötigen
import torch.nn.functional as F # Für die Berechnung des "Fehlers"
from torch.utils.data import DataLoader, Dataset #zur Vorbreitung der Daten für das Training in Mini-Batches

## Datensatz importieren und etwas Analyse

Wir werden mit einem Datensatz über gefälschte und echte Banknoten arbeiten. Hier wurden bereits Werte der Geldscheine aus den Bildern der Geldscheine extrahiert, mit denen wir arbeiten können, um einen Geldschein als echt oder gefälscht zu klassifizieren. Normalerweise wäre die Extraktion dieser Eigenschaften ein weitere Schritt, den man vorher durchführen müsste.

In [65]:
url = "https://archive.ics.uci.edu/static/public/267/banknote+authentication.zip"

df = pd.read_csv(url, header=None)
df.columns = ["x1", "x2", "x3", "x4", "class"]
df.head()

,x1,x2,x3,x4,class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [66]:
df.info() #Einige Eigenschaften des Datensatzes werden zusammengefasst

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1372 entries, 0 to 1371
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      1372 non-null   float64
 1   x2      1372 non-null   float64
 2   x3      1372 non-null   float64
 3   x4      1372 non-null   float64
 4   class   1372 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 53.7 KB


In [67]:
df.describe()

,x1,x2,x3,x4,class
count,1372.000000,1372.000000,1372.000000,1372.000000,1372.000000
mean,0.433735,1.922353,1.397627,-1.191657,0.444606
std,2.842763,5.869047,4.310030,2.101013,0.497103
min,-7.042100,-13.773100,-5.286100,-8.548200,0.000000
25%,-1.773000,-1.708200,-1.574975,-2.413450,0.000000
50%,0.496180,2.319650,0.616630,-0.586650,0.000000
75%,2.821475,6.814625,3.179250,0.394810,1.000000
max,6.824800,12.951600,17.927400,2.449500,1.000000


In [68]:
X = df.drop(columns="class") #Features
y = df["class"] #Target (echter oder gefälschter Geldschein)

In [69]:
y.value_counts(normalize=True) #leicht unausgeglichen

class
0    0.555394
1    0.444606
Name: proportion, dtype: float64

## Vorbereitung der Daten und des Modells (ohne Normalisierung/Standardisierung der Daten)

In [70]:
X = X.values
y = y.values

class BankDataset(Dataset):
    def __init__(self, X, y):
        self.features = torch.tensor(X, dtype=torch.float32)
        self.labels = torch.tensor(y, dtype=torch.float32)

    def __getitem__(self, index):
        x = self.features[index]
        y = self.labels[index]
        return x, y

    def __len__(self):
        return self.labels.shape[0]

In [71]:
train_size = int(0.8*X.shape[0]) # 80% des Datensatzes für das Training
val_size = X.shape[0]-train_size #20% zur Validierung des Modells

In [72]:
dataset = BankDataset(X, y)

torch.manual_seed(1) # um Ergebnisse zu replizieren

train_data, val_data = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(
    train_data,
    batch_size=16,
    shuffle=True
)

val_loader = DataLoader(
    val_data,
    batch_size=16,
    shuffle=False
)

In [74]:
# implementiere das Modell

class LogisticRegression(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.linear = nn.Linear(in_features=self.in_features, out_features=self.out_features)

    def forward(self, x):
        return self.linear(x)

In [75]:
model = LogisticRegression(4,1)
torch.sigmoid(model(torch.from_numpy(X).to(dtype=torch.float32)[:3])).view(-1)

tensor([0.0323, 0.0202, 0.6790], grad_fn=<ViewBackward0>)

In [79]:
def accuracy(model, dataloader):
    correct = 0
    total = 0
    model = model.eval()
    for (features, y_true) in dataloader:
        with torch.inference_mode():
            logits = model(features)
        probs = torch.sigmoid(logits).view(-1)
        y_pred = torch.where(probs>0.5,1,0)

        compare = y_pred == y_true
        correct += torch.sum(compare)
        total += len(compare)
    return correct/total

In [88]:
# Training-Schleife

epochs = 5

model = LogisticRegression(X.shape[1], 1)

loss_fn = F.binary_cross_entropy_with_logits
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.1)

history = {
    "train_losses": [],
    "val_acc":[]
}

for epoch in range(epochs):
    model = model.train()

    for batch_idx, (features, y_true) in enumerate(train_loader):
        logits = model(features)
        loss = loss_fn(logits, y_true.view(logits.shape))
        history["train_losses"].append(loss)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        history["val_acc"].append(accuracy(model, val_loader))

        if not batch_idx%40:
            print(f"Epoch: {epoch+1}/{epochs}",
                  f"Batch: {batch_idx+1}/{len(train_loader)}",
                  f"Train Loss: {history['train_losses'][-1]:.3f}",
                  f"Validation acc: {history['val_acc'][-1]:.3f}",
                  20*"-",
                  sep="\n")

Epoch: 1/5
Batch: 1/69
Train Loss: 0.586
Validation acc: 0.793
--------------------
Epoch: 1/5
Batch: 41/69
Train Loss: 0.055
Validation acc: 0.989
--------------------
Epoch: 2/5
Batch: 1/69
Train Loss: 0.097
Validation acc: 0.993
--------------------
Epoch: 2/5
Batch: 41/69
Train Loss: 0.001
Validation acc: 0.975
--------------------
Epoch: 3/5
Batch: 1/69
Train Loss: 0.070
Validation acc: 0.993
--------------------
Epoch: 3/5
Batch: 41/69
Train Loss: 0.009
Validation acc: 0.989
--------------------
Epoch: 4/5
Batch: 1/69
Train Loss: 0.087
Validation acc: 0.996
--------------------
Epoch: 4/5
Batch: 41/69
Train Loss: 0.111
Validation acc: 0.993
--------------------
Epoch: 5/5
Batch: 1/69
Train Loss: 0.099
Validation acc: 0.993
--------------------
Epoch: 5/5
Batch: 41/69
Train Loss: 0.035
Validation acc: 0.993
--------------------
